In [1]:
from util import *

with open('raw/tracks_uri_2_id.json') as f:
    tracks_uri_2_id=json.load(f)
with open('raw/tracks_id_2_uri.json') as f:
    tracks_id_2_uri=json.load(f)


playlists_tra,playlists_tst,playlist_tracks_tra,playlist_tracks_tst,tracks = load_data()

ptracks=pd.concat([playlist_tracks_tra,playlist_tracks_tst])
playlists=sorted(ptracks.pid.unique())
pid_2_ix={p:i for i,p in enumerate(playlists)}
track_ids=sorted(ptracks.track_id.unique())
tid_2_ix={t:i for i,t in enumerate(track_ids)}

r=np.zeros((ptracks.shape[0],))
c=np.zeros((ptracks.shape[0],))
d=np.ones((ptracks.shape[0],))
for i in tnrange(ptracks.shape[0]):
    r[i]=pid_2_ix[ptracks.iat[i,0]]
    c[i]=tid_2_ix[ptracks.iat[i,2]]
plays=csr_matrix((d,(r,c)),shape=(len(playlists),len(track_ids)))
plays

HBox(children=(IntProgress(value=0, max=66627428), HTML(value='')))

/home/cq/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)


<1009000x2262292 sparse matrix of type '<class 'numpy.float64'>'
	with 65743457 stored elements in Compressed Sparse Row format>

In [29]:
class Als:
    
    def __init__(self,alpha=0.01,epsilon=100,factors=5,max_iter=100):
        self.rc=500
        
        self.alpha=alpha
        self.epsilon=epsilon
        self.factors=factors
        self.max_iter=max_iter
        
              
    def fit(self,r):
        
        f_users=np.random.rand(r.shape[0],self.factors)
        f_items=np.random.rand(r.shape[1],self.factors)

        last_cost = np.inf    
        for it in tnrange(self.max_iter):
            a = np.dot(f_items.T,f_items)+self.alpha*np.eye(self.factors)
            b = np.dot(csr_matrix(f_items.T),r.T)
            f_users=np.dot(np.linalg.inv(a), b.toarray()).T
            
            a = np.dot(f_users.T,f_users)+self.alpha*np.eye(self.factors)
            b = np.dot(csr_matrix(f_users.T),r)
            f_items=np.dot(np.linalg.inv(a), b.toarray()).T
            
#             u = np.sum(np.sum(f_users**2))
#             m = np.sum(np.sum(f_items**2))
#             cost = self.alpha*(u + m)
            
#             for i in tnrange(0,f_users.shape[0],100):
#                 r_pred=np.dot(f_users[i],f_items.T)
#                 cost += np.sum(np.sum(r[i].toarray()-r_pred)**2)
                    
#             print('iter %d, cost %.2f'%(it+1,cost))

#             if np.isnan(cost) or (last_cost > cost and last_cost-cost < self.epsilon) or last_cost<cost:
#                 self.f_users=f_users
#                 self.f_items=f_items
#                 break

#             last_cost = cost
              
            self.f_users=f_users
            self.f_items=f_items
            
              
        return self

    def predict(self,y_tst,items,uid_2_ix):
        y_pred={}

        users=sorted(y_tst.pid.unique())
        for i in tnrange(len(users)):
            u = users[i]
            f_user=self.f_users[uid_2_ix[u]]
            r_items=np.dot(f_user,self.f_items.T)
            i_pred=np.argsort(r_items)
            
            seed_items=set(y_tst[y_tst.pid==u].track_id)
            pred_items=np.zeros((self.rc,))
            k=0
            for j in range(len(i_pred)-1,0,-1):
                itm=items[i_pred[j]] 
                if itm not in seed_items:
                    pred_items[k] = itm
                    k+=1
                    if k == self.rc:
                        break
            assert k==self.rc
            
            y_pred[u]=pred_items
                    
        return y_pred
    
als=Als(alpha=1e-1,factors=100,max_iter=2,epsilon=1e2)
als.fit(plays)
y_prd1=als.predict(playlist_tracks_tst,track_ids,pid_2_ix)
  
# als=Als(alpha=1e-1,factors=100,max_iter=2,epsilon=1e2)
# als.f_users=np.random.rand(plays.shape[0],als.factors)
# als.f_items=np.random.rand(plays.shape[1],als.factors)
# als.predict(playlist_tracks_tst,track_ids,pid_2_ix)

HBox(children=(IntProgress(value=0, max=2), HTML(value='')))

/home/cq/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)


HBox(children=(IntProgress(value=0, max=9000), HTML(value='')))

In [41]:
als.f_items[2]

array([-1.19601540e-04, -5.36585788e-05,  2.21614612e-04,  7.82722259e-05,
       -1.25002229e-04,  1.69030511e-04,  1.98427572e-04, -5.14854390e-05,
        9.84773176e-05, -1.97222193e-04, -1.42595329e-05,  4.69355773e-05,
       -1.06979894e-04,  1.63617415e-05, -9.11343243e-06,  7.56245859e-05,
       -9.54931704e-05,  8.24792690e-05,  1.48848086e-04,  1.86747976e-04,
       -1.88715189e-04,  2.71085282e-04, -1.05273094e-04,  3.20626591e-04,
        1.32804452e-04,  2.42191225e-05, -2.04538311e-04, -3.89814120e-04,
       -1.13859424e-04, -6.24053900e-05, -7.67824377e-05,  1.38484473e-04,
       -1.36450425e-04, -2.57563224e-04,  4.96399228e-06,  1.49748086e-05,
        2.69616369e-04, -1.33798089e-04,  2.10983563e-04,  1.54168311e-04,
       -9.33167475e-05, -2.88230321e-05,  1.15848623e-04,  2.14549385e-04,
        1.06303217e-04,  5.20417721e-05,  2.71809533e-04, -1.73990793e-04,
       -2.13365813e-04,  3.11983409e-04,  2.16167777e-04, -1.93023390e-04,
       -2.29769744e-04, -

/home/cq/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)


In [30]:
import title

playlists_tra['title']=playlists_tra.name.apply(title.build_title)
playlists_tst['title']=playlists_tst.name.apply(title.build_title)
playlists_tra.index=playlists_tra.pid
playlist_tracks_tra.index=playlist_tracks_tra.pid

p_no_title=set(playlists_tst.pid)-set(playlist_tracks_tst.pid)
y_prd2=title.recommend(playlists_tra,playlists_tst[playlists_tst.pid.apply(lambda p:p in p_no_title)],
                         playlist_tracks_tra,playlist_tracks_tst,tracks)
y_prd=y_prd1.copy()
y_prd.update(y_prd2)



HBox(children=(IntProgress(value=0, max=1000), HTML(value='')))

/home/cq/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)


NameError: name 'y_true' is not defined

In [32]:
results=np.zeros((len(y_prd),501),dtype='int32')
pids=sorted(list(y_prd.keys()))
for i,p in enumerate(pids):
    results[i][0]=p
    results[i,1:]=y_prd[p]
    
np.save('output/als_0518',results)

/home/cq/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)


In [ ]:

# class WAls(Als):
    
#     def __init__(self,alpha=0.01,epsilon=100,factors=5,max_iter=100,beta=1):
#         super().__init__(alpha,epsilon,factors,max_iter)
#         self.beta=beta
              
#     def fit(self,r,w):
        
#         self.f_users=np.random.rand(r.shape[0],self.factors)
#         self.f_items=np.random.rand(r.shape[1],self.factors)

#         last_cost = np.inf    
#         for it in range(self.max_iter):
#             for j in range(self.f_users.shape[0]):
#                 u_w=np.diag(w[j].toarray())
#                 print(u_w.shape)
#                 a=np.dot(self.f_items.T, np.dot(u_w,self.f_items))+self.alpha*np.eye(self.factors)
#                 b=np.dot(self.f_items.T, np.dot(u_w,r[j].toarray().T))
#                 self.f_users[j] = np.linalg.solve(a,b).T
            
            
#             for j in range(self.f_items.shape[0]):
#                 i_w=np.diag(w.T[j].toarray())
                    
#                 a=np.dot(self.f_users.T, np.dot(i_w,self.f_users))+self.alpha*np.eye(self.factors)
#                 b=np.dot(self.f_users.T, np.dot(i_w,r.T[j].toarray()))
#                 self.f_items[j] = np.linalg.solve(a,b).T
            
#             u = np.sum(np.sum(self.f_users**2))
#             m = np.sum(np.sum(self.f_items**2))
#             cost = self.alpha*(u + m)
            
#             for i in range(self.f_users.shape[0]):
#                 r_pred=np.dot(self.f_users[i],self.f_items.T)
#                 cost += np.sum(np.sum(np.multiply(w[i],r[i].toarray()-r_pred)**2))
                    
#             print('iter %d, cost %.2f'%(it+1,cost))

#             if np.isnan(cost) or (last_cost > cost and last_cost-cost < self.epsilon) or last_cost<cost:
#                 break

#             last_cost = cost
                
              
#         return self

#     def predict(self,y_tst,items,uid_2_ix):
#         y_pred=[]

#         users=sorted(y_tst.pid.unique())
#         for i,u in enumerate(users):
#             f_user=self.f_users[uid_2_ix[u]]
#             r_pred=sorted([(i,r) for i,r in enumerate(np.dot(f_user,self.f_items.T))],key=lambda x:x[1],reverse=True)
            
#             seed_items=set(y_tst[y_tst.pid==u].track_id)
#             pred_items=[]
#             k=0
#             for j,r in enumerate(r_pred):
#                 itm=items[r[0]] 
#                 if itm not in seed_items:
#                     pred_items.append(itm)
#                     k+=1
#                     if k == self.rc:
#                         break
#             assert k==self.rc
#             y_pred.append(pred_items)
                    
#         return y_pred
            
# beta=10
# weights=csr_matrix((d*beta,(r,c)),shape=(len(playlists),len(tracks)),)
# print(weights[0].toarray().shape)
# wals=WAls(alpha=1e-1,beta=beta,factors=10,max_iter=100,epsilon=1e2)
# wals.fit(plays,weights)
# y_prd=wals.predict(playlist_tracks_tst,tracks,pid_2_ix)
# print('r_score %.3f'%r_prec(y_true[10:],y_prd))